<a href="https://colab.research.google.com/github/Smbatus/Skillfactory_rds/blob/master/Project_7_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# подготавливаем данные

# для Google Colab:
!gsutil cp gs://oleg-zyablov/skillfactory/sf-dl-car-classification.zip .
!unzip sf-dl-car-classification.zip
!unzip -q -o sf-dl-car-classification/train.zip
!unzip -q -o sf-dl-car-classification/test.zip
sample_submission_path = '/content/sf-dl-car-classification/sample-submission.csv'
train_path = '/content/train/'
sub_path = '/content/test_upload/'

# для Kaggle Kernel
# !mkdir /kaggle/temp #папка для временных файлов
# !unzip /kaggle/input/train.zip -d /kaggle/temp
# !unzip /kaggle/input/test.zip -d /kaggle/temp
# sample_submission_path = '/kaggle/input/sample-submission.csv'
# train_path = '/kaggle/temp/train/'
# sub_path = '/kaggle/temp/test_upload/'
# %cd /kaggle/working  #переходим в рабочую директорию

#имена классов
class_names = [
  'Приора', #0
  'Ford Focus', #1
  'Самара', #2
  'ВАЗ-2110', #3
  'Жигули', #4
  'Нива', #5
  'Калина', #6
  'ВАЗ-2109', #7
  'Volkswagen Passat', #8
  'ВАЗ-21099' #9
]

Copying gs://oleg-zyablov/skillfactory/sf-dl-car-classification.zip...
/ [1 files][  1.6 GiB/  1.6 GiB]   41.0 MiB/s                                   
Operation completed over 1 objects/1.6 GiB.                                      
Archive:  sf-dl-car-classification.zip
   creating: sf-dl-car-classification/
  inflating: sf-dl-car-classification/sample-submission.csv  
  inflating: sf-dl-car-classification/train.csv  
 extracting: sf-dl-car-classification/test.zip  
 extracting: sf-dl-car-classification/train.zip  


In [2]:
# импорты

import os, re, math, random, time, gc, string, pickle, shutil, pathlib, itertools
import numpy as np, pandas as pd, matplotlib.pyplot as plt, PIL

import tensorflow as tf
#import tensorflow_addons as tfa
from tensorflow import keras
from tensorflow.keras import *
from tensorflow.keras.activations import *
from tensorflow.keras.applications import *
from tensorflow.keras.callbacks import *
from tensorflow.keras.layers import *
from tensorflow.keras.layers.experimental.preprocessing import *
from tensorflow.keras.losses import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.optimizers.schedules import *
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from PIL import Image
from IPython.display import clear_output
from tqdm.notebook import tqdm

import tensorflow.keras.backend as K
import tensorflow.keras.layers as L

from tensorflow.keras.layers.experimental.preprocessing import *

import imgaug.augmenters as iaa
import albumentations as A


In [3]:
image_size = (350, 350)
batch_size = 32

train_datagen = ImageDataGenerator(
    #rescale=1./255,
    validation_split=0.3,
    # ниже параметры аугментаций:
    horizontal_flip=True,
    width_shift_range=0.15, 
    height_shift_range=0.15,
    shear_range=0.15, 
    zoom_range=0.15, 
    rotation_range=15,
    brightness_range=(0.8, 1.2),
    zca_whitening=True, 
    zca_epsilon=1e-06,
    
)

val_datagen = ImageDataGenerator(
    #rescale=1/255,
    validation_split=0.2,
)

sub_datagen = ImageDataGenerator(
    #rescale=1/255
)

train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True,
    seed=0,
    subset='training'
)

val_generator = val_datagen.flow_from_directory(
    train_path,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True,
    seed=0,
    subset='validation'
)

sub_generator = sub_datagen.flow_from_dataframe( 
    dataframe=pd.read_csv(sample_submission_path),
    directory=sub_path ,
    x_col="Id",
    y_col=None,
    class_mode=None,
    target_size=image_size,
    batch_size=batch_size,
    shuffle=False
)

/usr/local/lib/python3.7/dist-packages/keras_preprocessing/image/image_data_generator.py:337: UserWarning: This ImageDataGenerator specifies `zca_whitening`, which overrides setting of `featurewise_center`.
  warnings.warn('This ImageDataGenerator specifies '


Found 10897 images belonging to 10 classes.
Found 3109 images belonging to 10 classes.
Found 6675 validated image filenames.


In [4]:
# Создадим базовую модель.
base_model = EfficientNetV2B3(include_top=False, input_shape=(350, 350, 3))

# Создадим голову.
head = Sequential([
    
  GlobalMaxPool2D(),
  Dropout(0.5),
  BatchNormalization(),
    
  Dense(256, activation='relu'),
  Dropout(0.25),
  BatchNormalization(),
    
  Dense(10, activation='softmax')
])

52617216/52606240 [==============================] - 1s 0us/step


In [5]:
model = Model(inputs=base_model.input, outputs=head(base_model.output))

In [6]:
# Компилируем модель.
model.compile(loss="categorical_crossentropy", 
              optimizer=optimizers.Adam(ExponentialDecay(1e-3, 100, 0.9)), 
              metrics=["accuracy"])



In [7]:
# Создаем обьект который будет сохранять лучшую модель.
my_callback = ModelCheckpoint(
    'best_model_batch.h5',
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    verbouse = 1,
    save_best_only=True)

In [ ]:
#Model.load_weights(filepath, by_name=False, skip_mismatch=False, options=None)

In [8]:
# обучаем модель с замороженными слоями.
model.fit(train_generator, validation_data=val_generator, callbacks=my_callback , epochs=20)

/usr/local/lib/python3.7/dist-packages/keras_preprocessing/image/image_data_generator.py:720: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
/usr/local/lib/python3.7/dist-packages/keras_preprocessing/image/image_data_generator.py:739: UserWarning: This ImageDataGenerator specifies `zca_whitening`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


Epoch 1/20
341/341 [==============================] - 676s 2s/step - loss: 0.7609 - accuracy: 0.7405 - val_loss: 0.1852 - val_accuracy: 0.9296
Epoch 2/20
341/341 [==============================] - 639s 2s/step - loss: 0.2116 - accuracy: 0.9253 - val_loss: 0.1331 - val_accuracy: 0.9527
Epoch 3/20
341/341 [==============================] - 627s 2s/step - loss: 0.1395 - accuracy: 0.9508 - val_loss: 0.1089 - val_accuracy: 0.9620
Epoch 4/20
341/341 [==============================] - 626s 2s/step - loss: 0.0975 - accuracy: 0.9638 - val_loss: 0.1105 - val_accuracy: 0.9620
Epoch 5/20
341/341 [==============================] - 627s 2s/step - loss: 0.0680 - accuracy: 0.9753 - val_loss: 0.0975 - val_accuracy: 0.9694
Epoch 6/20
341/341 [==============================] - 620s 2s/step - loss: 0.0566 - accuracy: 0.9799 - val_loss: 0.0916 - val_accuracy: 0.9698
Epoch 7/20
341/341 [==============================] - 611s 2s/step - loss: 0.0433 - accuracy: 0.9836 - val_loss: 0.0984 - val_accuracy: 0.9701

In [ ]:
model.load_weights('/content/best_model_batch.h5', by_name=False, skip_mismatch=False, options=None)

In [ ]:
# Сохраняем submission
predictions = model.predict(sub_generator, verbose=1)
predictions = predictions.argmax(axis=1)

submission = pd.DataFrame({
    'Id': sub_generator.filenames,
    'Category': predictions
}, columns=['Id', 'Category'])
submission.to_csv('submission.csv', index=False)

# Далее просто примеры кода который показал худший результат.

## Fine Tuning

In [ ]:
## Создадим модель.
##tune_model = Model(inputs=base_model.input, outputs=head(base_model.output))

In [ ]:
#len(tune_model.trainable_variables)

368

In [ ]:
## Замораживаем слои базовой модели.
#base_model.trainable = False

In [ ]:
## Компилируем модель.
#tune_model.compile(loss="categorical_crossentropy", 
#              optimizer=optimizers.Adam(ExponentialDecay(1e-3, 100, 0.9)), 
#              metrics=["accuracy"])


In [ ]:
## Можем посмотреть какие слои тренируемые.
#for layer in tune_model.layers:
#    print(layer, layer.trainable)

In [ ]:
## Создаем обьект который будет сохранять лучшую модель.
#my_callback = ModelCheckpoint(
#    'best_model_batch.h5',
#    save_weights_only=True,
#    monitor='val_accuracy',
#    mode='max',
#    verbouse = 1,
#    save_best_only=True)

In [ ]:
## обучаем модель с замороженными слоями.
#tune_model.fit(train_generator, validation_data=val_generator, callbacks=my_callback , epochs=10)

In [ ]:
## Сохраняем submission
#predictions = tune_model.predict(sub_generator, verbose=1)
#predictions = predictions.argmax(axis=1)
#
#submission = pd.DataFrame({
#    'Id': sub_generator.filenames,
#    'Category': predictions
#}, columns=['Id', 'Category'])
#submission.to_csv('submission.csv', index=False)

209/209 [==============================] - 92s 423ms/step


In [ ]:
## Размораживаем слои базовой модели.
#base_model.trainable = True

In [ ]:
## Размораживаем половину слоев базовой модели.
#fine_tune_at = len(base_model.layers)//2
#
#for layer in base_model.layers[:fine_tune_at]:
#    layer.trainable = False

In [ ]:
## Перекомпилируем модель с очень маленьким learning_rate.
#tune_model.compile(loss="categorical_crossentropy", 
#              optimizer=keras.optimizers.Adam(learning_rate=0.0001),
#              metrics=["accuracy"])

In [ ]:
## обучаем модель с размороженными слоями.
#tune_model.fit(train_generator, validation_data=val_generator, callbacks=my_callback , epochs=10)

In [ ]:
## Размораживаем все слои базовой модели и снова запускаем обучение.
#base_model.trainable = True

## Аугментации с albumentations

## Было опробованно некоторое количество различных комбинаций но давало стабильно плохой результат.


In [ ]:
#def augment(image):
#  #image = image.astype(np.uint8)
#  aug = A.Compose([
#    A.GaussNoise(always_apply=False, p=1.0, var_limit=(275, 380)),
#    A.HorizontalFlip(p=0.5),
#    A.Rotate(limit=30, interpolation=1, border_mode=4, value=None, mask_value=None, always_apply=False, p=0.5),
#
#    A.OneOf([
#    A.RandomBrightnessContrast(brightness_limit=0.3, contrast_limit=0.3),
#    A.RandomBrightnessContrast(brightness_limit=0.1, contrast_limit=0.1)],p=0.5),
#    A.GaussianBlur(p=0.05),
#    A.HueSaturationValue(p=0.5),
#    A.RGBShift(p=0.5),
#])  
#  return aug(image=image)['image']
#
#my_datagen = ImageDataGenerator(
#    preprocessing_function=augment
#)
#
#for _ in range(3):
#  my_generator = my_datagen.flow_from_directory(
#      train_path,
#      target_size=(384, 512),
#      batch_size=32,
#      shuffle=True,
#      seed=0
#  )
#  #show_first_images(my_generator)
#
#

In [ ]:
#image_size = (350, 350)
#batch_size = 32
#
#
#train_datagen = ImageDataGenerator(
#    #rescale=1./255,
#    validation_split=0.2,
#    # Здесь аугментации из albumentations
#    preprocessing_function=augment
#)
#
#
#val_datagen = ImageDataGenerator(
#    #rescale=1/255,
#    validation_split=0.2,
#)
#
#sub_datagen = ImageDataGenerator(
#    #rescale=1/255
#)
#
#train_generator = train_datagen.flow_from_directory(
#    train_path,
#    target_size=image_size,
#    batch_size=batch_size,
#    class_mode='categorical',
#    shuffle=True,
#    seed=0,
#    subset='training'
#)
#
#val_generator = val_datagen.flow_from_directory(
#    train_path,
#    target_size=image_size,
#    batch_size=batch_size,
#    class_mode='categorical',
#    shuffle=True,
#    seed=0,
#    subset='validation'
#)
#
#sub_generator = sub_datagen.flow_from_dataframe( 
#    dataframe=pd.read_csv(sample_submission_path),
#    directory=sub_path ,
#    x_col="Id",
#    y_col=None,
#    class_mode=None,
#    target_size=image_size,
#    batch_size=batch_size,
#    shuffle=False
#)

In [ ]:
## Создаем обьект который будет сохранять лучшую модель.
#my_callback = ModelCheckpoint(
#    'best_model_batch.h5',
#    save_weights_only=True,
#    monitor='val_accuracy',
#    mode='max',
#    verbouse = 1,
#    save_best_only=True)

In [ ]:
## строим модель
#model = Sequential([
#  EfficientNetV2B3(weights='imagenet', input_shape=(*image_size, 3), include_top=False), #предобученная нейросеть из модуля keras.applications
#  
#  GlobalMaxPool2D(),
#  Dropout(0.5),
#  BatchNormalization(),
#    
#
#
#  Dense(256, activation='relu'),
#  Dropout(0.25),
#  BatchNormalization(),
#  
#  Dense(10, activation='softmax')
#])
#
#model.compile(
#    loss=CategoricalCrossentropy(from_logits=True),
#    optimizer=Adam(ExponentialDecay(1e-3, 100, 0.9)),
#    metrics='accuracy'
#)
#

In [ ]:
## обучаем модель
#model.fit(train_generator, validation_data=val_generator, callbacks=my_callback , epochs=10)

## Здесь опробованы возможности аугментации Keras

In [ ]:
#from tensorflow.keras.layers.experimental.preprocessing import *
#
## Создаем обьект с аугментации.
#preprocessing_model = Sequential([
#                                  
#  RandomCrop(299, 299),
#  RandomFlip(mode='horizontal'),
#  RandomRotation(0.1),
#  RandomContrast(factor = (0.2, 0.3)),
#  RandomZoom(height_factor = (0.2, 0.3), width_factor = (0.2, 0.3),fill_mode="reflect", interpolation="bilinear"),
#  RandomWidth(factor = (0.2, 0.3) , interpolation="bilinear"),
#  RandomHeight(factor = (0.2, 0.3) , interpolation="bilinear"),
#
#
#])
#
## Создаем модель.
#model = Sequential([
#    
#  # добавляем аугментации в модель.  
#  preprocessing_model,
#  EfficientNetV2B3(weights='imagenet', input_shape=(*image_size, 3), include_top=False), #предобученная нейросеть из модуля keras.applications
#  
#  GlobalMaxPool2D(),
#  Dropout(0.5),
#  BatchNormalization(),
#    
#  Dense(256, activation='relu'),
#  Dropout(0.25),
#  BatchNormalization(),
#  
#  Dense(10, activation='softmax')
#
#])

In [ ]:
## Визуализация аугментаций
#for _ in range(3):
#  train_generator = train_datagen.flow_from_directory(
#      train_path,
#      target_size=(450, 600),
#      batch_size=1,
#      class_mode='categorical',
#      shuffle=True,
#      seed=0,
#      subset='training'
#  )
#  
#  def augment_batch_to_visualize(X, y):
#    return preprocessing_model(X, training=True).numpy(), y
#  
#  show_first_images((augment_batch_to_visualize(X, y) for X, y in train_generator))

In [ ]:
## Компилируем модель
#model.compile(
#    loss=CategoricalCrossentropy(from_logits=True),
#    optimizer=Adam(ExponentialDecay(1e-3, 100, 0.9)),
#    metrics='accuracy'
#)

In [ ]:
# обучаем модель
#model.fit(train_generator, validation_data=val_generator, callbacks=my_callback , epochs=10)